# **Assignment 3**


### Insert your details below. You should see a green checkmark.

In [ ]:
# Input student details (replace these with actual input values)
student = {
    "name": "",                    # Replace with your name
    "email": "@ualberta.ca",       # Replace with your email
    "ccid": "",                    # Replace with your CCID
    "idnumber": 1,                 # Replace with your ID number
}

In [ ]:
# Define the default and user-provided student dictionaries
def_student = {
    "name": "NAME as in eclass",
    "email": "UofA Email",
    "ccid": "CCID",
    "idnumber": 0
}


# Validation checks
assert set(def_student.keys()) == set(student.keys()),   "You don't have all the right entries! Make sure you have `name`, `email`, `ccid`, `idnumber`. ❌"
assert not any(value == "" for value in student.values()), "You haven't filled in all your details! No field should be empty. ❌"
assert all(isinstance(student[k], type(def_student[k])) for k in def_student),    "Your types seem to be off: `name::String`, `email::String`, `ccid::String`, `idnumber::Int`. ❌"
assert student["email"].endswith("@ualberta.ca"), "Your email must end with '@ualberta.ca'. ❌"

print(f"Welcome {student['name']}! ✅")


# Preamble

- Loading Packages
- Generating Utilities

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
import pandas as pd
from itertools import product



# Toy Data

Let's first start by importing a dataset for use throughout the assignment.
Later in the assignment, we will be using the full dataset for evaluating our models.
But for debugging purposes, we will start with a small subset of the data to test our models very quickly.


In [ ]:
def isclose(a, b, prec=1e-8):
    return np.linalg.norm(a - b) < prec


In [ ]:
def one_hot(x):
    # Collect unique values in order of appearance
    unique_vals = []
    for v in x:
        if v not in unique_vals:
            unique_vals.append(v)
    unique_vals = np.array(unique_vals)

    # Create a Boolean matrix of shape (num_unique, len(x))
    onehot_matrix = (unique_vals[:, None] == x[None, :]).astype(np.float32)

    # Transpose so that each row corresponds to one sample:
    # Final shape becomes (len(x), num_unique)
    return onehot_matrix.T


In [ ]:
def prepMNIST(samples: int):

    # Load the MNIST dataset.
    (x_train, y_train), _ = mnist.load_data()

    # print("Shape of x_train:", x_train.shape)
    # print("Shape of y_train:", y_train.shape)
    x_train = x_train.astype(np.float32) / 255.0

    x_train = x_train.reshape(x_train.shape[0], 28 * 28)

    # One-hot encode the labels.
    y_train = one_hot(y_train)

    return x_train[:samples], y_train[:samples]


In [ ]:
toy_x, toy_y = prepMNIST(500)

In [ ]:
print(toy_x.shape)

In [ ]:
print(toy_y.shape)

# Supervised Autoencoders

In this part of the notebook, you will implement a supervised autoencoder with a single hidden layer.
You will use backpropagation in order to take gradients w.r.t. the weights for a joint loss function:
$$
\ell(\mathbf{x}, \mathbf{y}; \mathbf{W}^{(2)}, \mathbf{W}_x^{(1)}, \mathbf{W}_y^{(1)}) = \ell_y(\hat{\mathbf{y}}, \mathbf{y}) + \beta \ell_x(\hat{\mathbf{x}}, \mathbf{x})
$$

where $\beta \ge 0$ is a hyperparameter that determines with how much weight the autoencoder portion of the SAE influences the hidden dimension.

As outlined in the assignment pdf, we will use identity activations (no activation, resulting in a linear function) for every layer except the heads corresponding to $\hat{\mathbf{y}}$. For this supervised part, because we are doing multinomial logistic regression, we use a softmax activation.
The loss $\ell_y$ is the multinomial cross-entropy for the "$\hat{\mathbf{y}}$" head of the neural network and the loss $\ell_x$ is the mean squared error for the "$\hat{\mathbf{x}}$" head, where $\hat{\mathbf{y}} = \text{softmax}(\mathbf{x} \mathbf{W}^{(2)} \mathbf{W}_y^{(1)})$ and $\hat{\mathbf{x}} = \mathbf{x} \mathbf{W}^{(2)}  \mathbf{W}_x^{(1)}$.

In [ ]:
class SAE:
    def __init__(self, features, hidden, outputs, rng):
        self.W2 = rng.normal(0, 1 / (features + hidden), (features, hidden))
        self.Wx = rng.normal(0, 1 / (features + hidden), (hidden, features))
        self.Wy = rng.normal(0, 1 / (hidden + outputs), (hidden, outputs))


In [ ]:
class SAEParams:
    def __init__(self, β=None, alpha=None, epochs=None, hidden=None, vals=None):
        if vals is not None:
            self.β, self.alpha, self.epochs, self.hidden = vals
        else:
            self.β = β
            self.alpha = alpha
            self.epochs = epochs
            self.hidden = hidden




## Utility functions
Here, we implement a few utility functions which we will need throughout the notebook.

In [ ]:
def softmax(X):
    exp_X = np.exp(X)
    return exp_X / np.sum(exp_X, axis=1, keepdims=True)


In [ ]:
def forward(sae, X):
    h = np.dot(X, sae.W2)
    X_hat = np.dot(h, sae.Wx)
    z = np.dot(h, sae.Wy)
    Y_hat = softmax(z)
    return X_hat, Y_hat

In [ ]:
def oneHotMax(X):
    def _inner(a):
        idx = np.argmax(a)
        out = np.zeros_like(a, dtype=int)
        out[idx] = 1
        return out

    # Apply _inner function to each row of the matrix
    return np.vstack([_inner(row) for row in X])


## Gradient Computation
In this section, we will implement the gradient of the SAE loss function. The loss function is defined at the beginning of the _Supervised Autoencoders_ section.


In [ ]:
def gradient(sae, params, X, Y):
    # Compute the gradient of the SAE parameters given inputs (X, Y)

    h = np.dot(X, sae.W2)

    # samples x features matrix
    X_hat = np.dot(h, sae.Wx)

    # samples x outputs matrix
    z = np.dot(h, sae.Wy)
    Y_hat = softmax(z)

    # Gradient for the Wx head
    delta_wx = params.β * (X_hat - X)  # (samples, features)
    grad_Wx = np.dot(h.T, delta_wx)  # (hidden, features)


    ### BEGIN SOLUTION
    # Gradient for the Wy head


	  ### END SOLUTION



    ### BEGIN SOLUTION
    # Gradient for W2

	  ### END SOLUTION

    return grad_Wx, grad_Wy, grad_W2


In [ ]:
# ##############
# Test Block
# ##############


# Let's first double check that you are getting the right shapes
sae = SAE(28*28, 128, 10, np.random.RandomState(1))  # Using RandomState as RNG
params = SAEParams(0.5, 0.001, 30, 128)

# Assuming the toy_x and toy_y are defined somewhere (as input data)
delWx, delWy, delW2 = gradient(sae, params, toy_x, toy_y)

# Check if the shapes of the gradients are correct
assert (delWx.shape == (128, 28*28)) and (delWy.shape == (128, 10)) and (delW2.shape == (28*28, 128)), "Incorrect shape of the gradient. ❌"


rng = np.random.RandomState(367)

sae = SAE(3, 4, 2, np.random.RandomState(2))
params = SAEParams(2.0, 0.01, 30, 4)

x = np.ones((5, 3))
y = rng.binomial(1, 0.5, (5, 2))  # Random binary values for y (Bernoulli distribution)

delWx, delWy, delW2 = gradient(sae, params, x, y)

# Expected values for the gradients
expect_delWx = np.array([
    [ 4.54203257,  4.25261002,  5.2015941 ],
    [ 2.51137886,  2.35135146,  2.87606336],
    [ 1.50355791,  1.40774979,  1.72189385],
    [-3.73462787, -3.4966539,  -4.27694386]
    ])
expect_delWy = np.array([
        [-0.61700488, -0.31675102],
        [-0.3411541 , -0.17513785],
        [-0.20424833, -0.10485471],
        [ 0.50732433,  0.26044445]
])
expect_delW2 = np.array([
[ 0.43764503, -1.2914451,   2.0948465,   1.65034573],
[ 0.43764503, -1.2914451,   2.0948465,   1.65034573],
[ 0.43764503, -1.2914451,   2.0948465,   1.65034573]
])

# Compare the computed gradients with expected gradients
assert np.allclose(delWx, expect_delWx), "Incorrect values for delWx. ❌"
assert np.allclose(delWy, expect_delWy), "Incorrect values for delWy. ❌"
assert np.allclose(delW2, expect_delW2), "Incorrect values for delW2. ❌"


print("Gradient check passed successfully!✅")

## Training Loop

Here you wil implement the training loop for the SAE, where you do SGD for multiple epochs with a constant stepsize. Remember to randomly shuffle the data for each epoch. Here we simply use vanilla SGD, with 1 sample per update (a mini-batch of size 1) and use a constant stepsize (rather than RMSProp and Adam). The reason we do this is to let you focus on the gradients for the SAE, rather than the additional details of the optimizer. Further, for this model and dataset, this provides sufficiently good performance. Of course, you can always test for yourself the impact of using mini-batches and improved stepsizes. But for your submission please use vanilla SGD.

In [ ]:
def train(X, Y, params, seed):
    rng = np.random.RandomState(seed)
    samples, features = X.shape
    classes = Y.shape[1]

    # Build the SAE model with N hidden units
    sae = SAE(features, params.hidden, classes, rng)

    # Train the model using SGD for params.epochs number of epochs
    # Use only a single sample for each iteration (stochastic gradient descent)
    # Remember to randomly shuffle the samples for each epoch
    for _ in range(params.epochs):
        ### BEGIN SOLUTION
        # Randomly shuffle the samples using rng permution
        sample_indices =

        for sample in sample_indices:


            # Get the single sample

            # Compute the gradients

            # Update the weights using gradient descent

        ### END SOLUTION

    return sae


In [ ]:
# ##############
# Test Block
# ##############

# Set random seed for reproducibility
rng = np.random.RandomState(1234)

# Create synthetic data
n = 20  # Number of samples
d = 5   # Number of features
m = 2   # Number of classes

# Generate two groups of data with different means
X1 = rng.normal(0.0, 1.0, (n // 2, d))
X2 = rng.normal(5.0, 1.0, (n // 2, d))

# Combine the two groups and shuffle the data
X = np.vstack((X1, X2))
perm = rng.permutation(n)
X = X[perm, :]

# Normalize the features
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# Generate labels (1 for X1, 2 for X2) and one-hot encode them
Y = np.hstack((np.ones(n // 2), 2 * np.ones(n // 2)))
Y = np.eye(m)[Y.astype(int) - 1]  # One-hot encode the labels

# Set hyperparameters
params = SAEParams(0.1, 0.1, 10, 10)

# Train the model
sae = train(X, Y, params, 1234)

# Check the learned weights against expected values

t1_c = sae.Wx[:2, :]
# Checking the first 2 rows of sae.Wx (hidden units x features)
t1 = np.allclose(t1_c,
  [[-0.0621649, -0.19708555, -0.24813198, -0.14997647, -0.1899449],
  [ 0.01971644,  0.12749735,  0.04233966,  0.11630627,  0.15949385]],
                  atol=1e-6)

t2_c =  sae.Wy[:5, :]
# Checking the first 5 rows of sae.Wy (hidden units x output classes)
t2 = np.allclose(t2_c,
  [[ 0.17618548, -0.10470722],
  [ 0.14259881, -0.07685819],
  [ 0.15505535, -0.00107455],
  [ 0.01042294, -0.11451652],
  [ 0.11057633, -0.11528749]],
                  atol=1e-6)

t3_c= sae.W2[1:3, 4:8]

# Checking the first 2 rows of sae.W2 (features x hidden units)
t3 = np.allclose(t3_c,
  [[ 0.23729971,  0.40224271, -0.147294,    0.09083973],
  [ 0.00425958, -0.39550697,  0.16109506, -0.09772874]],
                  atol=1e-6)


# Check if all tests pass
assert t1, "Incorrect values for Wx. ❌"
assert t2, "Incorrect values for Wy. ❌"
assert t3, "Incorrect values for W2. ❌"

print("Training test passed successfully!✅")


# Evaluating our model

Before we can do anything else (tuning hyperparameters, reporting statistics about our model, etc.), we must first decide how our model will be evaluated. In class, we primarily focused on classification accuracy. But other metrics are often used for classification, that better reflect how well the model performed for different classes and if it is skewed towards predicting one class more than others. These metrics can provide a more nuanced picture of the model.

To start, we will treat each class separately: for example, what is the true positive rate for the digit 2?



## True/False Positives/Negatives

To begin, let's implement fuctions which count the true/false positives and negatives. A _true positive_ is a prediction that was predicted positive when it has a postive label (which we represent as a 1). A _false positive_ is a prediction that was predicted positive when is has a negative label. A _true negative_ is a prediction that was predicted negative when it has a negative label (which we represent as a 0). A _false negative_ is a prediction that was predicted negative when is has a positive label. We provide the code for true positives. It is your job to complete the rest.

In [ ]:
# True positive
def tp(y, y_hat):
    return np.sum(y_hat[y == 1])

# True negative
def tn(y, y_hat):
    ### BEGIN SOLUTION
    return
    ### END SOLUTION

# False positive
def fp(y, y_hat):
    ### BEGIN SOLUTION
    return
    ### END SOLUTION

# False negative
def fn(y, y_hat):
    ### BEGIN SOLUTION
    return
    ### END SOLUTION


In [ ]:
# ##############
# Test Block
# ##############

__y = np.array([1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1])
__y_hat = np.array([0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0])

assert tp(__y, __y_hat) == 3, "Wrong tp number: ❌"
assert tn(__y, __y_hat) == 4, "Wrong tn number: ❌"
assert fp(__y, __y_hat) == 6, "Wrong fp number: ❌"
assert fn(__y, __y_hat) == 8, "Wrong fn number: ❌"

print("Tests passed successfully!✅")



Great! You've implemented four functions to count the number of:

* True positives
* True negatives
* False positives
* False negatives

## Evaluation criteria

Often, we'd like to consider the rates of true positives/negatives and false positives/negatives. Additionally, we often want to consider the accuracy of our predictions as well as the _precision_ and _recall_, which are defined as:

$$
\text{Precision: } \quad pr(y, \hat y) = \frac{TP}{TP + FP}
$$

$$
\text{Recall: } \quad rc(y, \hat y) = \frac{TP}{TP + FN}
$$

These definitions extend even to the multiclass setting. We can compute the precision and recall for each class. With precision, we get a sense of: when we classified an item as class j, was it actually class j? Precision is high when we are careful about assigning class j to an item. For example, imagine there are 100 samples that are class j. Even if we only label 10 of these as class j and do not incorrectly label any other items as class j, then our Precision = 1 which is maximal (here TP = 10 and FP = 0). Mentally, we can replace the term TP with Correctly-Labeled-as-Class-j and FP with Incorrectly-Labeled-as-Class-j.

Recall asks: did we find all the items that were labeled as class j? Recall can be high when precision is low. For example, if we label all items in our dataset as class j, then we would have Recall = 1 (False Negatives = 0). We can think of FN as Incorrectly-Not-Labeled-as-Class-j or Should-Have-Been-Labeled-as-Class-j. If we label everything as class j, then there are no items that Should-Have-Been-Labeled-as-Class-j, since it was already labeled class j. Of course, this labelling will have bad Precision. Ideally, we want to get high Recall and high Precision.

Reporting more metrics gives a multi-faceted view on your learned predictor. We will report all of them here, so you can see what they would look like. We have implemented some of the below functions for you: True Positive Rate, Accuracy and Error. It is your job to fill in the rest. Again, you will see checkmarks beside the metrics once they are implemented correctly.

* True positive rate
* False positive rate
* True negative rate
* False negative rate
* Precision
* Recall
* Accuracy
* Error


In [ ]:
# True positive rate (Recall)
def tpr(y, y_hat):
    return tp(y, y_hat) / (tp(y, y_hat) + fn(y, y_hat))

# False positive rate
def fpr(y, y_hat):
    ### BEGIN SOLUTION
    return
    ### END SOLUTION

# True negative rate
def tnr(y, y_hat):
    ### BEGIN SOLUTION
    return
    ### END SOLUTION

# False negative rate
def fnr(y, y_hat):
    ### BEGIN SOLUTION
    return
    ### END SOLUTION

# Precision
def pr(y, y_hat):
    ### BEGIN SOLUTION
    return
    ### END SOLUTION

# Recall
def rc(y, y_hat):
    ### BEGIN SOLUTION
    return
    ### END SOLUTION

# Accuracy
def accuracy(y, y_hat):
    return np.sum(y == y_hat) / len(y)

# Error
def error(y, y_hat):
    return 1 - accuracy(y, y_hat)


In [ ]:
# ##############
# Test Block
# ##############

assert np.isclose(tpr(__y, __y_hat), 0.2727272727272727),"True positive rate: ❌"
assert np.isclose(fnr(__y, __y_hat), 0.7272727272727273), "False positive rate: ❌"
assert np.isclose(tnr(__y, __y_hat), 0.4), "True negative rate: ❌"
assert np.isclose(fpr(__y, __y_hat), 0.6), "False negative rate: ❌"
assert np.isclose(pr(__y, __y_hat), 0.3333333333333333), "Precision: ❌"
assert np.isclose(rc(__y, __y_hat), 0.2727272727272727), "Recall: ❌"
assert np.isclose(accuracy(__y, __y_hat), 0.3333333333333333), "Accuracy: ❌"
assert np.isclose(error(__y, __y_hat), 0.6666666666666667), "Error: ❌"


print("All metrics have passed! ✅")


In [ ]:
# Initialize parameters for SAE
params = SAEParams(0.5, 0.01, 5, 16)

sae = train(toy_x, toy_y, params, 0)

# Forward pass to get predictions
_, Y_hat = forward(sae, toy_x)


# Convert probabilities to one-hot encoding by maximizing predictions
Y_hat = oneHotMax(Y_hat)


# Initialize arrays to hold the metrics for each class (10 classes)
col_tp = np.zeros(10, dtype=int)
col_tn = np.zeros(10, dtype=int)
col_fp = np.zeros(10, dtype=int)
col_fn = np.zeros(10, dtype=int)
col_tpr = np.zeros(10)
col_fnr = np.zeros(10)
col_tnr = np.zeros(10)
col_fpr = np.zeros(10)
col_pr = np.zeros(10)
col_rc = np.zeros(10)
col_acc = np.zeros(10)
col_err = np.zeros(10)

# Labels for each digit
labels = ["Zero", "One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine"]

# Loop through each class (0 to 9) and calculate metrics
for class_idx in range(10):
    col_tp[class_idx] = tp(toy_y[:, class_idx], Y_hat[:, class_idx])
    col_tn[class_idx] = tn(toy_y[:, class_idx], Y_hat[:, class_idx])
    col_fp[class_idx] = fp(toy_y[:, class_idx], Y_hat[:, class_idx])
    col_fn[class_idx] = fn(toy_y[:, class_idx], Y_hat[:, class_idx])
    col_tpr[class_idx] = tpr(toy_y[:, class_idx], Y_hat[:, class_idx])
    col_fnr[class_idx] = fnr(toy_y[:, class_idx], Y_hat[:, class_idx])
    col_tnr[class_idx] = tnr(toy_y[:, class_idx], Y_hat[:, class_idx])
    col_fpr[class_idx] = fpr(toy_y[:, class_idx], Y_hat[:, class_idx])
    col_pr[class_idx] = pr(toy_y[:, class_idx], Y_hat[:, class_idx])
    col_rc[class_idx] = rc(toy_y[:, class_idx], Y_hat[:, class_idx])
    col_acc[class_idx] = accuracy(toy_y[:, class_idx], Y_hat[:, class_idx])
    col_err[class_idx] = error(toy_y[:, class_idx], Y_hat[:, class_idx])

# Create a DataFrame with the results
df = pd.DataFrame({
    "Digit": labels,
    "True Positive": col_tp,
    "True Negative": col_tn,
    "False Positive": col_fp,
    "False Negative": col_fn,
    "True Positive Rate": col_tpr,
    "False Negative Rate": col_fnr,
    "True Negative Rate": col_tnr,
    "False Positive Rate": col_fpr,
    "Precision": col_pr,
    "Recall": col_rc,
    "Accuracy": col_acc,
    "Error": col_err
})

# Set display options for better tabular visualization with scrolling
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # No wrapping of columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_colwidth', None) # Show all the content without truncating

# Display the DataFrame as a proper table
display(df)  # For Jupyter notebook or IPython




# **Selecting Hyperparameters**

The supervised autoencoder class has 4 hyperparameters that need to be selected (for our implementation).
So far we have worked with toy data and a few hardcoded hyperparameters that we know work okay, but now it's time to start tuning our algorithm to get the best performance.

## **Internal K-Fold Cross-Validation**
In this section, you will implement **internal k-fold cross-validation** to select hyperparameters.


In [ ]:
def generate_param_settings(βs, αs, epochss, hiddens):
    # Create the Cartesian product of all parameters
    # This produces an iterator over tuples (β, α, epochs, hidden)
    param_combinations = product(βs, αs, epochss, hiddens)

    # For each tuple, create an SAEParams instance by passing the tuple to the constructor using the vals parameter
    return [SAEParams(vals=vals) for vals in param_combinations]


In [ ]:
def stack(arr):
    """
    Stacks matrices vertically.
    For example, if given matrices with shapes:
      A = (10, 23), B = (15, 23), C = (2, 23)
    then stack([A, B, C]) returns a (27, 23) matrix.
    """
    return np.vstack(arr)

def exclude(arr, idx):
    """
    Returns all elements of a list (or 1D array) except the element at the specified index.
    For example, if arr = [a, b, c, d] and idx = 2, this returns [a, b, d].
    """
    return [element for i, element in enumerate(arr) if i != idx]



We can choose to optimize any of the above metrics that we would like. By default, we have selected $\verb+accuracy+$. You can try other metrics, if you would like.
Below, we use `argmax` when selecting hyperparameters, because we want high accuracy (if we had errors, we would use `argmin`).

In [ ]:
def metric(sae, X, Y, f=accuracy):
    """
    Compute the average metric (e.g. accuracy) over each class.

    Parameters:
      sae : the model
      X   : input data as a 2D array (samples x features)
      Y   : true labels as a 2D array (samples x classes)
      f   : a function that computes a metric for two 1D arrays (default: accuracy)

    Returns:
      The mean of the metric computed for each class.
    """
    # Run forward propagation on the model
    # Assume forward(sae, X) returns (unused, Yhat)
    _, Yhat = forward(sae, X)

    # Apply one_hot_max to Yhat to get one-hot encoded predictions
    Yhat = oneHotMax(Yhat)

    # Determine the number of classes from Y's shape
    classes = Y.shape[1]

    # Initialize a metric value for each class
    metrics = np.zeros(classes)

    # Compute the metric for each class by comparing the true and predicted values column-wise
    for class_idx in range(classes):
        metrics[class_idx] = f(Y[:, class_idx], Yhat[:, class_idx])

    # Return the average metric across classes
    return np.mean(metrics)



Now you will implement `internal_cross_validation`.
Make sure that each of your `k` folds are non-overlapping and that every validation set is used exactly once.
We explicitly fix the random seed when training the model.
We want to reduce the number of confounding factors while selecting hyperparameters, for instance starting each neural network with the exact same set of weights. That way we are removing one potential source of stochasticity that could give noisier estimates of the performance for a hyperparameter.


In [ ]:
def internal_cross_validation(settings, k, X, Y, comparator=accuracy):
    """
    Performs internal cross validation over a list of hyperparameter settings.

    Parameters:
      settings   : list of SAEParams (or equivalent parameter settings)
      k          : number of folds to use for cross validation
      X          : input data (NumPy array of shape (samples, features))
      Y          : true labels (NumPy array of shape (samples, classes))
      comparator : function to compute a metric (default is accuracy)

    Returns:
      best_params: the hyperparameter setting (from settings) that achieves the best average metric
    """
    # Partition the data into k folds without shuffling
    Xs = []
    Ys = []

    ### BEGIN SOLUTION


    ### END SOLUTION

    # Use a fixed seed so every hyperparameter setting sees the same random seed
    seed = 0

    average_metrics = np.zeros(len(settings))

    ### BEGIN SOLUTION
    # Evaluate each parameter setting using k-fold cross validation
    for idx, params in enumerate(settings):



        for fold in range(k):
            # Create training data by stacking all folds except the current one



            # The validation data is the current fold



            # Train the model with the given parameter settings and seed


            # Compute the metric on the validation set


        # Store the average metric for the current parameter setting

    ### END SOLUTION


    best_idx = np.argmax(average_metrics)
    best_params = settings[best_idx]
    return best_params


In [ ]:
# #####################
# Test Block
# #####################

# Test block for internal cross validation

np.random.seed(367)

# Generate the hyperparameter settings.
settings = generate_param_settings([100, 1., 0.1],
                                    [0.001, 0.1],
                                    [1, 3, 5, 7],
                                    [2, 4, 6, 8])

# Generate synthetic data:
# __x is a 100 x 3 matrix drawn from Normal(0, 3)
__x = np.random.normal(0, 3, size=(100, 3))
# __y is a 100 x 2 matrix where each entry is drawn from a Bernoulli(0.5)
__y = np.random.binomial(1, 0.5, size=(100, 2)).astype(np.float32)

# Perform internal cross validation using 3 folds.
got = internal_cross_validation(settings, 3, __x, __y, comparator=accuracy)

# The expected best parameter
expected = SAEParams(0.1, 0.1 , 3, 4)

# Check if the returned parameter settings match the expected one.
# (Assuming SAEParams does not implement __eq__, we compare fields manually.)
is_equal = (got.β == expected.β and
            got.alpha == expected.alpha and
            got.epochs == expected.epochs and
            got.hidden == expected.hidden)


assert got.β == expected.β, "Wrong value for β: ❌"
assert got.alpha == expected.alpha, "Wrong value for alpha: ❌"
assert got.epochs == expected.epochs, "Wrong value for epochs: ❌"
assert got.hidden == expected.hidden, "Wrong value for hidden: ❌"


print("Internal cross validation test passed: ✅")


If all went well with your cross-validation code, then it will pass our tests and you should see a happy check mark above.

Now, let's run internal CV on our small MNIST dataset, to see which hyperparameter settings are picked.

In [ ]:
# Note: This cell may take several seconds to complete (e.g., ~30 seconds)

# Generate the hyperparameter settings.
settings = generate_param_settings(
    [1.0, 0.1],    # betas
    [0.01, 0.001], # alphas
    [2],           # epochs
    [16, 32]       # hidden units
)

# Of these parameter settings, let's see which one is the best on our small dataset.
# Note: This is only for a few hundred samples. Imagine running this for bigger models and datasets.
# Feel free to change the `comparator` parameter to try other metrics.
best_toy_param = internal_cross_validation(settings, 4, toy_x, toy_y, comparator=accuracy)

# Optionally, you can print the result to inspect the best parameter setting.
print("Best toy parameter setting:")
print("Best β", best_toy_param.β)
print("Best alpha", best_toy_param.alpha)
print("Best number epochs", best_toy_param.epochs)
print("Best number of hidden units", best_toy_param.hidden)



# **Evaluating Generalization Error**

Internal CV was used above to select hyperparameters $\verb+best-hypers+$ for the SAE. The model f learned with SAE+best-hypers on the entire dataset is the final model that we want to deploy. However, before deploying, we want to get a sense of its generalization performance (classification accuracy in deployment). In the notes we discussed using external CV, where we would split up the dataset into train/validate pairs like above, and then call SAE+internal CV on these train/validate pairs. This is called nested cross-validation: we have nested for loops where in the outer loop we iterate over the splits from external CV, and inside when we call SAE+internal CV, it splits the data further into multiple folds and loops over all hypers and those partitionings.

Unfortunately, this is very expensive. And already running on this MNIST dataset can take a bit too much compute for a student laptop. So, we are going to do a standard approximation used in practice, where we do a two-stage approach (see the notes for why this is more biased and can cause an overly optimistic estimate). Instead of using nest cross-validation to properly evaluate f, we will instead just call cross-validation on SAE+best-hypers. To implement cross-validation to evaluate SAE+best-hypers, we will use repeated random sampling (RRS). The main reason we use RRS instead of k-fold for our external CV is because it actually helps reduce bias a little bit from this un-nested procedure, and because it lets you implement the other way approach we discussed to generated partitions for CV. We will report the average and standard error of our accuracy across these partitions.

A few notes:
 * The train/test sets must be non-overlapping
 * Each train/test split needs to be sampled randomly `k` times
 * You should use `train_percent` percentage of the samples in the train set, and the remainder in the test set (i.e. if `train_percent=0.9`, then 90% of the samples are training samples and 10% are testing samples)

In [ ]:
def repeated_random_sampling(params, X, Y, k, train_percent, rng, comparator=accuracy):
    """
    Performs repeated random sampling cross validation.

    Parameters:
      params        : SAEParams instance containing hyperparameters.
      X             : NumPy array of shape (samples, features).
      Y             : NumPy array of shape (samples, classes).
      k             : Number of repetitions.
      train_percent : Fraction (float) of data to use for training.
      rng           : A NumPy random generator (e.g., np.random.default_rng(seed)).
      comparator    : A metric function (default: accuracy).

    Returns:
      A tuple (mean_metric, std_error) where:
        - mean_metric is the average metric across the k folds,
        - std_error is the standard error (std / sqrt(k)).
    """
    samples = X.shape[0]
    train_samples = int(np.floor(samples * train_percent))

    # Ensure consistent initialization across folds.
    seed = 0

    metrics = np.zeros(k)


    for fold in range(k):
        print("rrs", fold + 1, k)

        idxs = rng.permutation(samples)

        ### BEGIN SOLUTION

        # Rearrange X and Y according to the shuffled indices.



        # Split into training and test sets.



        # Train the model using the given parameters and fixed seed.


        # Compute the metric on the test set.


        ### END SOLUTION

    # Return the mean metric and the standard error.
    return np.mean(metrics), np.std(metrics) / np.sqrt(k)


In [ ]:
# Create the parameter setting
params = SAEParams(0.1, 0.001, 2, 10)

# Create a random generator with seed 1
rng = np.random.default_rng(1)

# Generate synthetic data:
# X: 500 x 30 matrix drawn from a Normal distribution with mean=1 and std=20
X = rng.normal(loc=1, scale=10, size=(500, 30))
# Y: 500 x 3 matrix with entries drawn from a Bernoulli(0.4) distribution
Y = rng.binomial(n=1, p=0.4, size=(500, 3)).astype(np.float32)

# Run repeated random sampling cross validation with 9 repetitions,
# using 42% of the samples for training.
acc, stderr = repeated_random_sampling(params, X, Y, 9, 0.42, rng, comparator=accuracy)

print(acc, stderr)
# Return True if the accuracy is at least 0.5 and the standard error is <= 0.01.
assert acc >= 0.5 and stderr <= 0.01, "Repeated random sampling test failed: ❌"

print("Repeated random sampling test passed:.✅")


Does RRS report the correct accuracy and standard error on a small toy dataset?

In [ ]:
# Create a random generator with seed 3
rng = np.random.default_rng(3)

# Call repeated_random_sampling with 10 repetitions and a training fraction of 0.90.
result = repeated_random_sampling(best_toy_param, toy_x, toy_y, 10, 0.90, rng)

print("Accuracy:", result[0], "Std Error:", result[1])

# Putting it all together

Finally, we will run the SAE with internal CV to get best-hypers on the full MNIST dataset and then run external CV to evaluate the model learned with those hypers. If you want to examine other metrics, you can change the metric used in RRS and the outputted numbers will be the mean and standard error across the folds for that metric.

In [ ]:
def run_experiment():
    # Load 10,000 MNIST samples
    X, Y = prepMNIST(5000)

    # Generate hyperparameter settings:
    # beta: [0.1, 0.0], alpha: [0.001, 0.01], epochs: [1, 2], hidden units: [50, 100]
    settings = generate_param_settings(
        [0.1, 0.0],     # beta
        [0.001, 0.01],  # alpha
        [1, 2],         # epochs
        [50, 100]       # hidden units
    )

    # Perform internal cross validation with 3 folds to select the best parameters.
    best_params = internal_cross_validation(settings, 3, X, Y)
    print("Best β: ", best_params.β)
    print("Best alpha: ", best_params.alpha)
    print("Best number of epochs: ", best_params.epochs)
    print("Best hidden units size: ", best_params.hidden)

    # Create a random generator with seed 734
    rng = np.random.default_rng(734)

    # Run repeated random sampling using the best parameters, 5 repetitions, and a training fraction of 0.75.
    acc, stderr = repeated_random_sampling(best_params, X, Y, 5, 0.75, rng)

    return acc, stderr

In [ ]:
## uncomment the following when you are ready to run it
## it will approximately 10~15m on colab

accuracy_val, std_err = run_experiment()
print("Repeated Random Sampling -> Accuracy:", accuracy_val, "Std Error:", std_err)